<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/main/DL_Project_wandb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

--2021-06-21 16:19:24--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.238.33
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.238.33|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip          11%[=>                  ]   8,90M   431KB/s    eta 2m 37s^C
[dataset.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  dataset.zip may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of dataset.zip or
        dataset.zip.zip, and cannot find dataset.zip.ZIP, period.


In [2]:
!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 85 (delta 37), reused 63 (delta 19), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [ ]:
!nvidia-smi

Tue May 18 17:50:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    #train_root = "/content/dataset/train",
    #test_root = "/content/dataset/test", 
    #queries_root = "/content/dataset/queries",
    #attributes_file = "/content/dataset/annotations_train.csv",
    train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    loss = "triplet / crossentropy, etc",
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=8,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9, 
    mAP_rank=10)

In [2]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from cost_functions import AttributesLossWrapper
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import FinetunedModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

[Errno 2] No such file or directory: '/content/deeplearning_unitn'
/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dl_unitn
cuda:0


In [3]:
!pip install wandb -q
import wandb
wandb.login()

/bin/bash: pip: command not found


ModuleNotFoundError: No module named 'wandb'

In [2]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    train_set, val_set, val_queries = split_data(config)
    
    model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
    #print(model)

    id_ground_truth_dict = build_ground_truth(val_set, val_queries)
    train(model, train_loader, val_loader, criterion, optimizer, config)

    #test(model, val_loader, val_queries_loader, config, save_model=True) #True to save model

    return model

In [3]:
def build_ground_truth(val_set, val_queries):
    values = []
    for q in val_queries:
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))

    #values = []
    #for q in val_queries:
    #    matches = []
    #    for v in val_set:
    #        if v.split("_")[0] == q.split("_")[0]:
    #            matches.append(v)
    #    value = set(matches)
    #    values.append(value)
    #ground_truth_dict = dict(zip(val_queries, values))

    return ground_truth_dict


In [4]:
def split_data(config):
    """Returns a list with the names of theimages in each set"""
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries

def setup(train_set, val_set, val_queries, config):
    #Create pytorch Datasets 
    composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            #full_train_set=full_train_set,
                            images_list=train_set,
                            transform=composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_set,
                         transform=composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_queries,
                         transform=composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=8)
                                               
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size=config["validation_batch_size"], 
                                             shuffle=False, 
                                             num_workers=8)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             batch_size=config["validation_batch_size"],
                                             shuffle=False, 
                                             num_workers=8)

    #model = ReIdModel(n_classes=len(train_dataset.classes)).to(device)
    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")
    model = FinetunedModel(architecture=config["backbone"] ,n_classes=attr_len).to(device)

    # Setup the optimization problem
    #criterion = cost_functions.cross_entropy()
    criterion = AttributesLossWrapper(2)
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, criterion, optimizer

In [5]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    if config["wandb"]:
        wandb.watch(model, criterion, log="all", log_freq=10)
    model.train()
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    for epoch in tqdm(range(config["epochs"])):
        for batch_idx, (inputs, identity, attributes) in enumerate(train_loader):
            loss, predicted = train_batch(inputs, identity, attributes, model, optimizer, criterion)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(inputs, identity, attributes, model, optimizer, criterion):
    inputs = inputs.to(device)
    attributes = attributes.to(device)
    
    # Forward pass
    outputs = model(inputs)
    # Apply the loss
    loss = criterion(outputs, attributes)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    # Zeros the gradient
    predicted = torch.round(outputs[0].squeeze(1))

    return loss, predicted

In [7]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [8]:
# This coulf be the attributes classification function
def test(model, loader, config, save_model=False):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for inputs, labels, attributes in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        if config["wandb"]:
            wandb.log({"test_accuracy": correct / total})
    if save_model :
      # Save the model in the exchangeable ONNX format
      torch.onnx.export(model, inputs, "model.onnx")
      if config["wandb"]:
        wandb.save("model.onnx")

In [9]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config)

{'wandb': False, 'device': 'auto', 'train_root': '/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train', 'test_root': '/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test', 'queries_root': '/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries', 'attributes_file': '/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv', 'dataset': 'Market1501', 'backbone': 'resnet18', 'loss': 'triplet / crossentropy, etc', 'split': {'full_training_size': 0.75, 'train_size': 0.8}, 'compose': {'resize_h': 224, 'resize_w': 224}, 'epochs': 1, 'training_batch_size': 8, 'validation_batch_size': 32, 'learning_rate': 0.01, 'weight_decay': 1e-06, 'momentum': 0.9, 'mAP_rank': 10}
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9862
Validation set size: 2783
Number of validation queries: 344
Number of attributes: 27
Backbone feature size: 512


  0%|          | 0/1 [00:00<?, ?it/s]

Loss after 00392 examples: 15.244
Loss after 00792 examples: 10.037
Loss after 01192 examples: 11.543
Loss after 01592 examples: 10.522
Loss after 01992 examples: 10.383


KeyboardInterrupt: 

In [5]:
train_set, val_set, val_queries = split_data(config)
model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
id_ground_truth_dict = build_ground_truth(val_set, val_queries)

Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9567
Validation set size: 3045
Number of validation queries: 377
Number of attributes: 27
Backbone feature size: 512


In [9]:
def get_features_from_loader(model, loader):
    model.eval()
    with torch.no_grad():
        all_features = torch.zeros(len(loader.dataset), model.feature_size)
        for batch_idx, (inputs, ids, attr) in tqdm(enumerate(loader)):
                inputs = inputs.to(device)
                features = model(inputs, get_features=True)
                for in_batch, f in enumerate(features):
                    all_features[(batch_idx * loader.batch_size) + in_batch] = f
        return all_features


def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """

    # Run the model on some test examples
    with torch.no_grad():
        
        # Compute the features for queries and gallery
        print("Computing gallery features...")
        gallery_features = get_features_from_loader(model, gallery_loader)
        print("Computing query features...")
        query_features = get_features_from_loader(model, queries_loader)
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        print("Computing cosine similarities...")
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0])
        for idx, q in enumerate(query_features):
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)
        
        print("Similarity matrix shape: " + str(sims_matrix.size()))
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])

        #Build the dictionary to compute the mAP
        predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
        mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
        
        print(f"mAP: {mAP}")
        if config["wandb"]:
            wandb.log({"mAP": mAP})
    if save_model :
      # Save the model in the exchangeable ONNX format
      torch.onnx.export(model, inputs, "model.onnx", export_params=True)
      if config["wandb"]:
        wandb.save("model.onnx")

In [10]:
test_mAP(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=False)

Computing gallery features...


0it [00:00, ?it/s]

Computing query features...


0it [00:00, ?it/s]

Computing cosine similarities...
Similarity matrix shape: torch.Size([368, 2973])
current_ground_truth_query: 0
current_ground_truth_query_set: {704, 1282, 2339, 2148, 2245, 1127, 1833, 2479, 1876, 471, 2714, 2719}
current_predictions_list: [959, 1194, 2037, 1286, 472, 33, 85, 2244, 602, 1711]
0.0
current_ground_truth_query: 1
current_ground_truth_query_set: {2564, 1286, 2827, 2060, 916, 2456, 1435, 1405, 1695, 1313, 2082, 2596, 421, 1325, 1070, 176, 1974, 1591, 568, 2682, 444, 2239, 1984, 2112, 1218, 1346, 582, 2375, 72, 200, 2253, 2941, 975, 2899, 342, 2006, 2008, 91, 2652, 2782, 996, 2664, 1385, 1769, 2286, 2805, 1274, 2171, 765}
current_predictions_list: [2941, 1218, 200, 1346, 371, 2456, 2003, 1153, 2914, 218]
0.09863945578231292
current_ground_truth_query: 2
current_ground_truth_query_set: {1793, 391, 2311, 15, 275, 1434, 1946, 668, 1309, 798, 2206, 418, 1323, 1069, 942, 1838, 2862, 948, 1851, 1596, 957, 317, 1985, 451, 2883, 1861, 1736, 1225, 2763, 81, 338, 1490, 2002, 2132, 277

In [12]:

def get_attributes_from_loader(model, loader):
    model.eval()

    all_predictions = np.empty(shape=[0, 27], dtype=np.byte)
    all_attrs = np.empty(shape=[0, 27], dtype=np.byte)

    with torch.no_grad():
        for batch_idx, (inputs, ids, attr) in tqdm(enumerate(loader)):
                inputs = inputs.to(device)
                outputs = model(inputs, get_features=False)
                #print("attr:",attr)
                predictions = torch.empty(attr.size()[1], attr.size()[0])
                for attr_idx, output in enumerate(outputs):
                    if output.size()[1] == 1: #If the output is binary
                        pred = torch.round(torch.squeeze(output, 1))
                    else: #Otherwise it is multiclass
                        pred = torch.argmax(output, dim=1)
                    predictions[attr_idx] = pred

                predictions = torch.transpose(predictions, 0, 1).cpu().numpy()
                attr = attr.cpu().numpy()

                all_predictions = np.append(all_predictions, predictions, axis=0)
                #print("all_predictions shape: ", all_predictions.shape)
                all_attrs = np.append(all_attrs, attr, axis=0)
                #print("all_attrs shape: ", all_attrs.shape)
        return all_predictions, all_attrs

def test_attributes(model, loader):
    predictions, attr = get_attributes_from_loader(model, val_loader)
    print("pred shape: ", predictions.shape)
    print("attr shape: ", attr.shape)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    for i in range(0, predictions.shape[1]):
        y_true, y_pred = attr[:, i], predictions[:, i]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        if i == 0: #If it is age
            precision_list.append(precision_score(y_true, y_pred, average='macro'))
            recall_list.append(recall_score(y_true, y_pred, average='macro'))
            f1_score_list.append(f1_score(y_true, y_pred, average='macro'))
        else:
            precision_list.append(precision_score(y_true, y_pred))
            recall_list.append(recall_score(y_true, y_pred))
            f1_score_list.append(f1_score(y_true, y_pred))

    average_acc = np.mean(accuracy_list)
    average_precision = np.mean(precision_list)
    average_recall = np.mean(recall_list)
    average_f1score = np.mean(f1_score_list)

    print("accuracy_list: ", accuracy_list)
    print("precision_list: ", precision_list)
    print("recall_list: ", recall_list)
    print("f1_score_list: ", f1_score_list)

    print("average_acc: ", average_acc)
    print("average_precision: ", average_precision)
    print("average_recall: ", average_recall)
    print("average_f1score: ", average_f1score)

    if config["wandb"]:
            wandb.log({"mAP": mAP})
            wandb.log({"accuracy_list": accuracy_list)
            wandb.log({"precision_list": precision_list)
            wandb.log({"recall_list": recall_list)
            wandb.log({"f1_score_list": f1_score_list)

            wandb.log({"average accuracy": average_acc)
            wandb.log({"average precision": average_precision)
            wandb.log({"average recall": average_recall)
            wandb.log({"average f1": average_f1score)

test_attributes(model, val_loader)

0it [00:00, ?it/s]

pred shape:  (3045, 27)
attr shape:  (3045, 27)
accuracy_list:  [0.02922824302134647, 0.6512315270935961, 0.33760262725779966, 0.896551724137931, 0.5366174055829228, 0.4663382594417077, 0.04893267651888342, 0.4541871921182266, 0.8686371100164204, 0.5054187192118227, 0.09490968801313629, 0.5300492610837438, 0.296551724137931, 0.3333333333333333, 0.11954022988505747, 0.12151067323481117, 0.5018062397372742, 0.7024630541871921, 0.586863711001642, 0.48407224958949097, 0.05779967159277504, 0.37701149425287356, 0.7954022988505747, 0.8088669950738916, 0.37766830870279144, 0.7428571428571429, 0.8036124794745484]
precision_list:  [0.2593101343101343, 0.20891364902506965, 0.27212988017008116, 0.0, 0.8457997698504027, 0.7572815533980582, 0.0, 0.2926613616268789, 0.08176100628930817, 0.46755921730175076, 0.08397200933022327, 0.35253456221198154, 0.06820319849482596, 0.05149181905678537, 0.10461229946524064, 0.10753768844221105, 0.08698583951449763, 0.036214953271028034, 0.4085714285714286, 0.05752